<a href="https://colab.research.google.com/github/vihan-lakshman/mutagenic/blob/main/progress_report_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.7 MB/s eta 0:00:00


In [5]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3-open").to("cuda") # or "cpu"

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

data/entry_list_safety_29026.list:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

data/ParentChildTreeFile.txt:   0%|          | 0.00/595k [00:00<?, ?B/s]

data/1utn.pdb:   0%|          | 0.00/569k [00:00<?, ?B/s]

...

esm3_function_decoder_v0.pth:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [7]:
import pandas as pd
test_data = pd.read_csv('/content/test-validation-dataset.csv')

In [8]:
def fill_sequence(sequence, del_start, del_end):
  masked_sequence = ""

  for i in range(len(sequence)):
    if i not in set(range(del_start, del_end+1)):
      masked_sequence += sequence[i]

    else:
      masked_sequence += "_"

  assert len(masked_sequence) == len(sequence)
  return masked_sequence

test_data['masked_sequence'] = test_data.apply(lambda x: fill_sequence(x['sequence'], x['del_start'], x['del_end']), axis=1)

In [9]:
import torch

def esm_generate(model, truncated_sequence, num_steps):
  generated_protein = ESMProtein(sequence=truncated_sequence)
  protein = model.generate(generated_protein, GenerationConfig(track="sequence", temperature=0.7))
  torch.cuda.empty_cache()
  return protein.sequence

from Bio import SeqIO, pairwise2, AlignIO

def calculate_distance(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2)
    alignment = alignments[0]  # Take the best alignment
    matches = sum(a == b for a, b in zip(alignment[0], alignment[1]))
    identity = matches / len(alignment[0])
    return 1 - identity  # Distance is 1 - identity

results = []

for index, row in test_data.iterrows():
  # Skip any sequence that is too long to fit in GPU memory
  if row['len'] > 1800:
    continue

  num_steps = row['masked_sequence'].count('_')
  generated_sequence = esm_generate(model, row['masked_sequence'], num_steps)
  score = calculate_distance(row['sequence'], generated_sequence)
  results.append(score)



/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
100%|██████████| 1/1 [00:05<00:00,  5.51s/it]
/usr/local/lib/python3.10/dist-packages/esm/pretrained.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will 

In [16]:
import numpy as np

np.mean(results)

0.24452280652747588